In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from pathlib import Path
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from sklearn.linear_model import LinearRegression
import seaborn as sns
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from scipy.signal import periodogram
import datetime as dt


In [2]:
import os
print(os.getcwd())
pd.set_option('mode.chained_assignment', None)


C:\Users\Lena\Dokumente\GitHub\DataLiteracy


In [3]:
path = "Daten/data.csv"
df =  pd.read_csv(path,header=1, sep=";", names=['date','Tunnel','Steinlach','Hirschau'], parse_dates=['date']).dropna()
df['date']=pd.to_datetime(df['date']).dt.date
df=df.groupby(['date']).sum()

display(df)
display(df.index)

,Tunnel,Steinlach,Hirschau
date,,,
2018-01-01,721.0,0.0,184.0
2018-01-02,1593.0,0.0,246.0
2018-01-03,1116.0,0.0,98.0
2018-01-04,1278.0,0.0,120.0
2018-01-05,1703.0,0.0,221.0
...,...,...,...
2022-11-23,4758.0,9211.0,779.0
2022-11-24,4336.0,8844.0,701.0
2022-11-25,4190.0,8394.0,592.0


Index([2018-01-01, 2018-01-02, 2018-01-03, 2018-01-04, 2018-01-05, 2018-01-06,
       2018-01-07, 2018-01-08, 2018-01-09, 2018-01-10,
       ...
       2022-11-18, 2022-11-19, 2022-11-20, 2022-11-21, 2022-11-22, 2022-11-23,
       2022-11-24, 2022-11-25, 2022-11-26, 2022-11-27],
      dtype='object', name='date', length=1792)

In [4]:
weeks = pd.date_range('2018-01-01', '2022-11-27', freq='W').to_numpy()
weeks= np.repeat(weeks, 7)

df_weeks = df.copy()
df_weeks['weekdate']= weeks.tolist()
df_weeks['weekdate']=pd.to_datetime(df_weeks['weekdate']).dt.date
df_weeks = df_weeks.groupby('weekdate').sum()
display(df_weeks)

,Tunnel,Steinlach,Hirschau
weekdate,,,
2018-01-07,8949.0,0.0,1422.0
2018-01-14,20909.0,0.0,2764.0
2018-01-21,16793.0,15439.0,1707.0
2018-01-28,20033.0,41665.0,2771.0
2018-02-04,19570.0,40661.0,2437.0
...,...,...,...
2022-10-30,34393.0,62741.0,7941.0
2022-11-06,22355.0,43893.0,4554.0
2022-11-13,31530.0,58626.0,5542.0


In [5]:
df.index = pd.to_datetime(df.index)
df['day'] = df.index.dayofweek
df['week'] = df.index.week
df['dayofyear'] = df.index.dayofyear
df['year'] = df.index.year

display(df)

C:\Users\Lena\AppData\Local\Temp\ipykernel_5912\1977790416.py:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df['week'] = df.index.week


,Tunnel,Steinlach,Hirschau,day,week,dayofyear,year
date,,,,,,,
2018-01-01,721.0,0.0,184.0,0,1,1,2018
2018-01-02,1593.0,0.0,246.0,1,1,2,2018
2018-01-03,1116.0,0.0,98.0,2,1,3,2018
2018-01-04,1278.0,0.0,120.0,3,1,4,2018
2018-01-05,1703.0,0.0,221.0,4,1,5,2018
...,...,...,...,...,...,...,...
2022-11-23,4758.0,9211.0,779.0,2,47,327,2022
2022-11-24,4336.0,8844.0,701.0,3,47,328,2022
2022-11-25,4190.0,8394.0,592.0,4,47,329,2022
